# Script to train ML algorithms after feature selection and feature extraction  except MARS

In [ ]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import pandas as pd
# import seaborn as ns
from sklearn.decomposition import PCA,FastICA,KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.ensemble import RandomForestRegressor
from genetic_selection import GeneticSelectionCV
import matplotlib.pyplot as plt
import math
import time
from RegscorePy.aic import aic # for calculating Akaike’s Information Criterion
from RegscorePy.bic import bic # for calculating Bayesian Information Criterion

In [ ]:
# %load MLOperationsUtilities.py
def readDataFromCsv(file):
    """ Read csv from files
    
    Parameters
    ----------
    file: str
        Filename to be read
    
    Returns
    ----------
    pandas.DataFrame
        Returns the dataframe containing the dataset
    """
    import pandas as pd
    print ("Reading the file from: ",file)
    df = pd.read_csv(file)
    return df

def loadDataset(paths=['../datasets/files_generated/Personality/study1_features_data.csv',
                      '../datasets/files_generated/Personality/study2_features_data.csv'],target='Neuroticism'):
    """ prepares the data and loads it
    
    Parameters
    ----------
    paths: array
        Filenames to be read
    target: str
        perosnality label to be specified
    
    Returns
    ----------
    pandas.DataFrame
        Returns the dataframe containing the dataset
    """
    for path in paths:
        if 'study1' in path:
            df = readDataFromCsv(path)
            df= df.select_dtypes (['int64','float64']).drop(['VP','age','user_id'],axis=1)
            print('The shape of the data  currently in study1: ',df.shape)
            X_study1,y_study1= df.drop(['Neuroticism', 'Extraversion', 
                                        'Openness', 'Agreeableness','Conscientiousness'],axis=1),df[target]
        elif 'study2'in path:
            df = readDataFromCsv(path)
            df = df.select_dtypes(['int64','float64']).drop(['user_id','UserId','VP','Age','Handedness_Score'],axis=1)
            print('The shape of the data  currently in study2: ',df.shape)
            X_study2,y_study2=df.drop(['Neuroticism', 'Extraversion', 'Openness', 'Agreeableness','Conscientiousness'],axis=1),df[target]
        else:
            df = pd.read_csv(path,index_col=0)
            X,y=df.drop(['Neuroticism', 'Extraversion', 
                                        'Openness', 'Agreeableness','Conscientiousness','user_id'],axis=1),df[target]
    # concat both the studies
    if(len(paths)>1):
        X = pd.concat([X_study1,X_study2])
        y= pd.concat([y_study1,y_study2])
    
    print('The shape of the data after concating both the studies {}'.format(X.shape))
    print('The shape of the target after concating both the studies {}'.format(y.shape))
    assert df.isnull().values.any()==False, 'Please check for null values'
    df_result={'data':X,'target':y}
    return df_result

In [ ]:
def optimalModelSelection(model,param_grid,X,y,method='grid'):
    '''Tune the hyperparameters to find the best score personality data'''
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import StandardScaler,RobustScaler
    from sklearn.pipeline import make_pipeline,Pipeline
    from sklearn.model_selection import KFold,GridSearchCV,RandomizedSearchCV
    
    K = 10
    kf = KFold(n_splits=K, shuffle=True,random_state=32)
    
    scoring={'r2':'r2','mse':'neg_mean_squared_error','mae':'neg_mean_absolute_error'}
    if(method=='grid'):
        search = GridSearchCV(model, param_grid, cv=kf,n_jobs=-1,scoring=scoring,return_train_score=True,refit='r2')
        search.fit(X,y)
    if(method=='random'):
        search=RandomizedSearchCV(estimator = model, param_distributions = param_grid, 
                               n_iter = 100, cv = kf, verbose=1, 
                               random_state=32, n_jobs = -1,scoring=scoring,return_train_score=True,refit='r2')
        search.fit(X,y)
    
    print('Best params: {}'.format(search.best_params_))
    print('RMSE: %0.2f'%(np.sqrt(-search.cv_results_['mean_test_mse'][search.best_index_])))
    print("R2(Validation): %0.2f (+/- %0.2f)" % (search.best_score_,search.cv_results_['std_test_r2'][search.best_index_]))
    print("R2(Train): %0.2f (+/- %0.2f)" % (search.cv_results_['mean_train_r2'][search.best_index_],
                                                 search.cv_results_['std_train_r2'][search.best_index_]))
    print("MAE(Validation): %0.2f (+/- %0.2f)" % (-search.cv_results_['mean_test_mae'][search.best_index_],
                                                  search.cv_results_['std_test_mae'][search.best_index_]))
    print("MAE(Train): %0.2f (+/- %0.2f)" % (-search.cv_results_['mean_train_mae'][search.best_index_],
                                                 search.cv_results_['std_train_mae'][search.best_index_]))
     
    return search.best_estimator_,search.best_params_, search.best_score_,search.cv_results_,search.best_index_

In [ ]:
def genetic_selection(estimator,X,y):
    '''
    Returns the selected columns after GA
    '''
    np.random.seed(10)
    from pyearth import Earth
    # calculate the optimal population size
    if (isinstance(estimator,Earth)==False):
#         population_size=math.ceil((267.43*np.log(X.shape[0]))-293.21) # reference from paper
        population_size=50
        generations=20
    else:
        population_size=100 
        generations=40 # this may not lead to optimal solution and may suffer from premature convergence
    selector = GeneticSelectionCV(estimator,
                                          cv=5,
                                          scoring="r2",
                                          n_population=population_size,
                                          crossover_proba=0.5,
                                          mutation_proba=0.01,
                                          n_generations=generations,
                                          tournament_size=3,
                                          caching=True,
                                          n_jobs=-1)
    start = time.time()
    selector = selector.fit(X, y)
    print("---Finished in %s seconds ---" % (np.round(time.time() - start,3)))
    print("Number of columns selected:",len(np.where(selector.support_==True)[0]))
    if isinstance(X, pd.DataFrame):
        columns=X.columns[selector.support_] # returns the column names
    else:
        columns=np.where(selector.support_==True) # return the indices of the numpy array
        
    return columns

In [ ]:
def perform_evaluation(path, estimator,param_grid,method='grid',transformation=False):
    if transformation==False:
        targets=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
    else:
        '''For transformation'''
        not_columns=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
        normality_test_features_path='/mnt/vdb1/Personality-Ratings/NormalityCheck/combined_univariate_normality_test_features_mahalanobis_transformed.csv'
#         normality_test_features_path='Tables/NormalityCheck/combined_univariate_normality_test_features_mahalanobis_transformed.csv'
        mahalanobis = pd.read_csv(normality_test_features_path)
        mahalanobis = list(mahalanobis[mahalanobis['Normality']==True]['Features'].values)

        for col in not_columns:
            if(col in mahalanobis):
                mahalanobis.remove(col)
        targets=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
        print("Columns that should not be selected are:",mahalanobis)

    results={}
    predictions={}
    for target in targets:
        target_result={}
        print('Performing prediction for target:',target)
        
        personality=loadDataset(paths=path,target=target)
        X=personality.get('data')
        y=personality.get('target')
        columns = X.loc[:, X.var() == 0.0].columns
        print("columns thrown away because they have 0 variance:",columns)
        X = X.loc[:, X.var() != 0.0]
        print("Shape of the data after removing 0 variance columns:",X.shape)
        if transformation==True:
            # apply only the normal columns
            X=X[mahalanobis]
            print("Shape of the data after selected transformed columns:",X.shape)
        
        # Create correlation matrix
        corr_matrix = X.corr().abs()

        # Select upper triangle of correlation matrix
        upper_traingle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

        # Find index of feature columns with correlation greater than or equal to 0.80
        to_drop_cols = [column for column in upper_traingle.columns if any(upper_traingle[column] >= 0.80)]
        X=X.drop(X[to_drop_cols],axis=1)
        
        print("Current Shape of data:",X.shape)
        
        # features selected
        selected_features= genetic_selection(estimator,X,y)
            
        # scale the data
        scaler=StandardScaler()
        X=scaler.fit_transform(X[selected_features])
            
        # tune hyperparameters on the optimal subset
        best_estimator_,best_params_, best_score_,cv_results_,best_index_= optimalModelSelection(estimator,param_grid,X,y,method=method)
        
        # calculate the AIC
        y_pred_train  = best_estimator_.fit(X,y).predict(X)

        # calculate MAPE
        mape_score_val = np.mean(np.abs((y - y_pred_train) / y)) * 100
        

        ''''Store the residuals in the table'''
        residuals = np.array(y)- y_pred_train
        #store it in a seperate table
        prediction= {'Original':np.array(y),'Predicted':y_pred_train,'Residuals':residuals}

        predictions[target]=prediction
            
        # append the results
        target_result['R2(Validation)']=best_score_
        target_result['Adjusted R2(Validation)']=1-(1-best_score_)*(len(y)-1)/(len(y)-len(selected_features)-1)
        target_result['StandardError(Validation)']=cv_results_['std_test_r2'][best_index_]
        target_result['RMSE(Validation)']=np.sqrt(np.abs(cv_results_['mean_test_mse'][best_index_]))
        target_result['R2(Train)']=cv_results_['mean_train_r2'][best_index_]
        target_result['RMSE(Train)']=np.sqrt(np.abs(cv_results_['mean_train_mse'][best_index_]))

        target_result['#Features']=len(selected_features)
        target_result['Features']=selected_features.values
        target_result['MAE(Validation)']= -cv_results_['mean_test_mae'][best_index_]
        target_result['MAE(Train)']= -cv_results_['mean_train_mae'][best_index_]
        target_result['MAPE(Validation)']=mape_score_val

        # store the result with respect to target
        results[target]=target_result
    
    return results,predictions

In [ ]:
def perform_evaluation_with_pca(path, estimator,param_grid,n_components,method='grid',transformation=False):
    if transformation==False:
        targets=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
    else:
        '''For transformation'''
        not_columns=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
        normality_test_features_path='/mnt/vdb1/Personality-Ratings/NormalityCheck/combined_univariate_normality_test_features_mahalanobis_transformed.csv'
        mahalanobis = pd.read_csv(normality_test_features_path)
        mahalanobis = list(mahalanobis[mahalanobis['Normality']==True]['Features'].values)

        for col in not_columns:
            if(col in mahalanobis):
                mahalanobis.remove(col)
        targets=['Neuroticism', 'Extraversion', 
                        'Openness', 'Agreeableness','Conscientiousness']
        print("Columns that should not be selected are:",mahalanobis)

    
    results={}
    predictions={}
    for target in targets:
        print(target)
        target_result={}
        print('Performing prediction for target:',target)
        
        personality=loadDataset(paths=path,target=target)
        X=personality.get('data')
        y=personality.get('target')
        columns = X.loc[:, X.var() == 0.0].columns
        print("columns thrown away because they have 0 variance:",columns)
        X = X.loc[:, X.var() != 0.0]
        print("Shape of the data after removing 0 variance columns:",X.shape)
        if transformation==True:
            # apply only the normal columns
            X=X[mahalanobis]
            print("Shape of the data after selected transformed columns:",X.shape)
        
        # Create correlation matrix
        corr_matrix = X.corr().abs()

        # Select upper triangle of correlation matrix
        upper_traingle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

        # Find index of feature columns with correlation greater than or equal to 0.80
        to_drop_cols = [column for column in upper_traingle.columns if any(upper_traingle[column] >= 0.80)]
        X=X.drop(X[to_drop_cols],axis=1)
        
        print("Current Shape of data:",X.shape)
        
            
        # scale the data
        scaler=StandardScaler()
        X=scaler.fit_transform(X)
        
        # perform PCA
        print('inside ga with pca function')
        print(type(n_components))
        print(n_components)
        pca= PCA(n_components=n_components)
        pca.fit(X)
        X=pca.transform(X)
        print('number of principal components:',pca.n_components_)
        
        # apply genetic algorithm to select the best PC
        selected_features= genetic_selection(estimator,X,y)
        
            
        # tune hyperparameters on the optimal subset
        best_estimator_,best_params_, best_score_,cv_results_,best_index_= optimalModelSelection(estimator,
                                                                                                 param_grid,
                                                                                                 X[:,selected_features[0]],y,method=method)
        
        y_pred_train  = best_estimator_.fit(X[:,selected_features[0]],y).predict(X[:,selected_features[0]])
        print(y_pred_train)
        # calculate MAPE
        mape_score_val = np.mean(np.abs((y - y_pred_train) / y)) * 100
       
        ''''Store the residuals in the table'''
        residuals = np.array(y)- y_pred_train
        #store it in a seperate table
        prediction= {'Original':np.array(y),'Predicted':y_pred_train,'Residuals':residuals}
        predictions[target]=prediction
            
        # append the results
        target_result['R2(Validation)']=best_score_
        target_result['Adjusted R2(Validation)']=1-(1-best_score_)*(len(y)-1)/(len(y)-X[:,selected_features[0]].shape[1]-1)
        target_result['StandardError(Validation)']=cv_results_['std_test_r2'][best_index_]
        target_result['RMSE(Validation)']=np.sqrt(np.abs(cv_results_['mean_test_mse'][best_index_]))
        target_result['R2(Train)']=cv_results_['mean_train_r2'][best_index_]
        target_result['RMSE(Train)']=np.sqrt(np.abs(cv_results_['mean_train_mse'][best_index_]))
        target_result['#Features']=len(selected_features[0])
        target_result['Features']=selected_features[0]
        target_result['MAE(Validation)']= -cv_results_['mean_test_mae'][best_index_]
        target_result['MAE(Train)']= -cv_results_['mean_train_mae'][best_index_]
        target_result['MAPE(Validation)']=mape_score_val
        
        # store the result with respect to target
        results[target]=target_result
    return results, predictions

In [ ]:
#create models
def runAllModels(path, filename,n_components=None,transformation=False,perform_pca=False):
    
    # random forest
    print('********Applying Random forest****************')
    rf = RandomForestRegressor(random_state=101)
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
    max_depth = [int(x) for x in np.linspace(1, 5, num = 5)]
    min_samples_split = [int(x) for x in np.linspace(10, 100, num = 10)]
    min_samples_leaf = [int(x) for x in np.linspace(10, 60, num = 20)]
    bootstrap = [True, False]
    max_features=['auto','sqrt']
    param_grid={'n_estimators': n_estimators,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
#                 'bootstrap': bootstrap,
                'max_features':max_features
               }
    if perform_pca==True:
        results_rf,predictions_rf= perform_evaluation_with_pca(path,rf,param_grid,n_components,method='random',transformation=transformation)
    else:
        results_rf,predictions_rf= perform_evaluation(path,rf,param_grid,method='random',transformation=transformation)            
    print(predictions_rf)
    np.random.seed(15)
    print('********Applying Support vector machine****************')
    from sklearn.svm import SVR
#     C_space=np.logspace(-1,0,10)
    C_space=np.logspace(-1,1,10)
#     epsilon_space= np.logspace(-1,0,10)
    epsilon_space= np.logspace(-1,0,10)
#     gamma_space = np.logspace(-3, 3, 10)
    gamma_space = np.logspace(-3, -2, 10)
    param_grid={'C':C_space,'epsilon':epsilon_space,'gamma':gamma_space}
    svr = SVR(kernel = 'rbf')
    if perform_pca==True:
        results_svm,predictions_svm = perform_evaluation_with_pca(path,svr,param_grid,n_components,method='random',transformation=transformation)
    else:
        results_svm,predictions_svm = perform_evaluation(path,svr,param_grid,method='random',transformation=transformation)

    print('********Applying Linear regression with stochastic gradient descent****************')
    from sklearn.linear_model import SGDRegressor
    param_grid={#'max_iter':[100,500,1000],
                'max_iter':[50,100],
                'penalty':[None],
                'eta0':[0.01,0.1,0.5]
               }
    sgd_reg = SGDRegressor(random_state=32)
    if perform_pca==True:
        print(n_components)
        print(type(n_components))
        results_sgd,predictions_sgd = perform_evaluation_with_pca(path,sgd_reg,param_grid,n_components,transformation=transformation)
    else:
        results_sgd, predictions_sgd = perform_evaluation(path,sgd_reg,param_grid,transformation=transformation)
    


    ## lasso regression
    print('********Applying Lasso Regression****************')
    from sklearn.linear_model import Lasso
#     alpha_space = np.logspace(-4, 0, 50)
    alpha_space = np.logspace(0, 1, 100)
    param_grid={'alpha':alpha_space}
    lasso = Lasso(random_state=32)
    if perform_pca==True:
        results_lasso, predictions_lasso = perform_evaluation_with_pca(path,lasso,param_grid,n_components,transformation=transformation)
    else:
        results_lasso, predictions_lasso = perform_evaluation(path,lasso,param_grid,transformation=transformation)
    

    ## elastic net 
    print('********Applying Elastic Net Regression****************')
    from sklearn.linear_model import ElasticNet
#     alpha_space = np.logspace(-4, 0, 50)
    alpha_space = np.logspace(0, 2 , 50)
    param_grid={'alpha':alpha_space}
    enet = ElasticNet(random_state=32)
    if perform_pca==True:
        results_enet,predictions_enet = perform_evaluation_with_pca(path,enet,param_grid,n_components,transformation=transformation)
    else:
        results_enet, predictions_enet = perform_evaluation(path,enet,param_grid,transformation=transformation)
    
    
    np.random.seed(32)
    #MARS
    print('********Applying MARS****************')
    from pyearth import Earth
    max_degree_space=[1]
    penalty_space=np.logspace(-1,1,20)
    minspan_alpha=np.logspace(-3,1,20)
    max_terms=[10,20,25]
    # endspan_alpha= np.linspace(0, 1.0, num = 10)
    # endspan=[5]
    param_grid={'max_degree':max_degree_space,
        'penalty':penalty_space,
                'use_fast':[True],
        'max_terms':max_terms
               }

    df_rf=pd.DataFrame(results_rf).T
    df_rf['Target']=df_rf.index
    df_rf=df_rf.reset_index(drop=True)
    df_rf['Algorithm']='Random Forest'
    df_rf.set_index(['Algorithm'])

    df_svm=pd.DataFrame(results_svm).T
    df_svm['Target']=df_svm.index
    df_svm=df_svm.reset_index(drop=True)
    df_svm['Algorithm']='SVM'
    df_svm.set_index(['Algorithm'])

    df_sgd=pd.DataFrame(results_sgd).T
    df_sgd['Target']=df_sgd.index
    df_sgd=df_sgd.reset_index(drop=True)
    df_sgd['Algorithm']='Linear regression'
    df_sgd.set_index(['Algorithm'])

    df_lasso=pd.DataFrame(results_lasso).T
    df_lasso['Target']=df_lasso.index
    df_lasso=df_lasso.reset_index(drop=True)
    df_lasso['Algorithm']='Lasso Regression'
    df_lasso.set_index(['Algorithm'])

    df_enet=pd.DataFrame(results_enet).T
    df_enet['Target']=df_enet.index
    df_enet=df_enet.reset_index(drop=True)
    df_enet['Algorithm']='Elastic Net'
    df_enet.set_index(['Algorithm'])

#     concat the df
    pd.concat([
        df_rf,df_svm,df_sgd,
        df_lasso,
        df_enet,
    ]).to_csv(filename,index=False)
    print("File saved")
    del df_rf,df_svm,df_sgd,df_lasso,df_enet,
    
    def createPredictionsTable(predictions):
        targets = ['Neuroticism','Extraversion', 'Openness', 'Agreeableness','Conscientiousness']
        df = pd.DataFrame()
        for target in targets:
            pq= pd.DataFrame(predictions.get(target))
            pq.rename(index=str, columns={"Original": "Original_"+target, "Prediction": "Prediction_"+target,
                                          'Residuals':'Residuals_'+target}, inplace=True)

            df = pd.concat([df,pq],axis=1)
        return df
    
    df_sgd=createPredictionsTable(predictions_sgd)
    df_lasso=createPredictionsTable(predictions_lasso)
    df_enet=createPredictionsTable(predictions_enet)
    df_svm=createPredictionsTable(predictions_svm)
    df_rf=createPredictionsTable(predictions_rf)
    if transformation==False and perform_pca==True:
        filename='feature_selection_alltargets_mahalanobis_PCA_'+str(n_components)+'_predictions.xlsx'
    elif transformation==False and perform_pca==False:
        filename='feature_selection_alltargets_mahalanobis'+'_predictions.xlsx'
    elif transformation==True and perform_pca==True:
        filename='feature_selection_alltargets_mahalanobis_transformed_PCA_'+str(n_components)+'_predictions.xlsx'
    else:
        filename='feature_selection_alltargets_mahalanobis_transformed'+'_predictions.xlsx'
       
    with pd.ExcelWriter(filename) as writer:  # doctest: +SKIP
        df_sgd.to_excel(writer, sheet_name='Linear Regression')
        df_lasso.to_excel(writer, sheet_name='Lasso Regression')
        df_enet.to_excel(writer, sheet_name='Elastic Net')
        df_svm.to_excel(writer, sheet_name='SVM')
        df_rf.to_excel(writer, sheet_name='Random Forest')
        print('File with filename %s saved successfully'%(filename))
    
    print('file saved sucessfully')

Study1 + Study2 original
---

In [ ]:
# evaluate model on original distribution
#/mnt/vdb1
if __name__=='__main__':
    paths=['/mnt/vdb1/datasets/files_generated/Personality/study1_features_data_out_mahalanobis.csv',
                      '/mnt/vdb1/datasets/files_generated/Personality/study2_features_data_out_mahalanobis.csv']
    filename='Tables/feature_selection_mahalanobis_alltargets.csv'
    runAllModels(paths,filename,transformation=False,perform_pca=False)

Study1  + Study2 Transformation
---

In [ ]:
# evaluate model data on transformed distribution features
if __name__=='__main__':
    path=['/mnt/vdb1/datasets/files_generated/Personality/combined_features_data_out_mahalanobis_transformedDistributions.csv']
    filename='Tables/feature_selection_mahalanobis_transformed_alltargets.csv'
    runAllModels(path,filename,transformation=True,perform_pca=False)